## File needs to be in the correct format and have the right columns
This code is to get probabilities from file [NearGalCat or GW_Query],then we make manual cuts to the file. We keep only the targets with the highest probability, then we calculate the dust and add a dust column. Then we make cuts to the file. Anything bigger than 0.5 we delete. Then we manually add a priority column which is the inverse of the probability and lastly add, using code, the Type, Exposure Time, and absolute magnitude. Also the aparent magnitude and 2D probability columns were deletd in the final file. Not sure if we will need them to calculate dynamic exposures.  

In [3]:
%matplotlib inline

import healpy as hp
import numpy as np
import json
import astropy.coordinates as coord
import astropy.units as u
import matplotlib.pyplot as plt
from astropy.io import fits
import csv
from scipy.special import erf

## Takes HEALPix file and converts it?

In [2]:
# ColDefs(
#     name = 'PROB'; format = '1024E'; unit = 'pix-1'
#     name = 'DISTMU'; format = '1024E'; unit = 'Mpc'
#     name = 'DISTSIGMA'; format = '1024E'; unit = 'Mpc'
#     name = 'DISTNORM'; format = '1024E'; unit = 'Mpc-2'
# )

hpx_file = 'LALInference_skymap.fits.gz,3'
# prob, header = hp.read_map(hpx_file, h=True)

# hpx_file = 'bayestar.fits.gz'
prob, distmu, distsigma, distnorm, header = hp.read_map(hpx_file, field=(0,1,2,3), h=True)

for h in header:
    print(h)

NSIDE = 256
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT
Ordering converted to RING
Ordering converted to RING
Ordering converted to RING
Ordering converted to RING
('XTENSION', 'BINTABLE')
('BITPIX', 8)
('NAXIS', 2)
('NAXIS1', 16384)
('NAXIS2', 768)
('PCOUNT', 0)
('GCOUNT', 1)
('TFIELDS', 4)
('TTYPE1', 'PROB')
('TFORM1', '1024E')
('TUNIT1', 'pix-1')
('TTYPE2', 'DISTMU')
('TFORM2', '1024E')
('TUNIT2', 'Mpc')
('TTYPE3', 'DISTSIGMA')
('TFORM3', '1024E')
('TUNIT3', 'Mpc')
('TTYPE4', 'DISTNORM')
('TFORM4', '1024E')
('TUNIT4', 'Mpc-2')
('PIXTYPE', 'HEALPIX')
('ORDERING', 'NESTED')
('COORDSYS', 'C')
('EXTNAME', 'xtension')
('NSIDE', 256)
('FIRSTPIX', 0)
('LASTPIX', 786431)
('INDXSCHM', 'IMPLICIT')
('OBJECT', '0')
('DATE-OBS', '2017-01-04T10:11:58.600292')
('MJD-OBS', 57757.42498379962)
('DATE', '2017-01-06T16:37:42.000000')
('CREATOR', 'run_sky_area')
('DISTMEAN', 922.3304700403676)
('DISTSTD', 258.1090838196292)


In [3]:
# prob, distmu, distsigma, distnorm, header = hp.read_map(hpx_file, field=(0,1,2,3), h=True)

# Degrees per pixel
npix = len(prob)
sky_area = 4 * 180**2 / np.pi
print("Sky Area per pix: %s" % (sky_area / npix))

# nside = the lateral resolution fo the HEALPix map
nside = hp.npix2nside(npix)
print("Resolution (nside): %s" % nside)

Sky Area per pix: 0.052455852825697924
Resolution (nside): 256


In [4]:
def validFloat(i):   #WHAT?!
    valid = True
    try:
        float(i)
    except ValueError:
        valid = False
    return valid

In [5]:
with open("GW_Query.csv","r") as csvinput:                   #Here I understand
    with open("20170314_GW_Resolved.csv","w") as csvoutput:
        
                reader = csv.reader(csvinput, delimiter=",")
                writer = csv.writer(csvoutput, lineterminator="\n")

                output_rows = []
                header_row = next(reader)
                header_row.append("Prob")
                header_row.append("DistMu")
                header_row.append("DistSigma")
                header_row.append("DistNorm")
                header_row.append("TotalSigma")
                header_row.append("z-Prob")
                header_row.append("3D-Prob")
                output_rows.append(header_row)

                # Remaing rows
                for row in reader:

                    ra = row[0] # RA (degree) col
                    dec = row[1] # DEC (degree) col
                    c = coord.SkyCoord(ra+dec,unit=(u.hourangle, u.deg),frame="fk5")

                    # Convert to spherical polar coords
                    theta = 0.5 * np.pi - np.deg2rad(c.dec.degree)
                    phi = np.deg2rad(c.ra.degree)

                    # Resolve RA, DEC => Pixel
                    px = hp.ang2pix(nside, theta, phi)

                    # Get corresponding HEALPIX quantities
                    p = prob[px]
                    du = distmu[px] 
                    ds = distsigma[px]
                    dn = distnorm[px]

                    row.append(p)

                    row.append(du)
                    row.append(ds)
                    row.append(dn)

                    # Compute composite probabilities
                    if (validFloat(row[18]) and validFloat(row[19])):

                        galaxy_distance = float(row[18])
                        galaxy_distance_err = float(row[19])

                        if (galaxy_distance > 0 and galaxy_distance_err > 0):
                            sigmaTotal = np.abs(galaxy_distance - du)/np.sqrt(galaxy_distance_err**2 + ds**2)
                            zProb = 1-erf(sigmaTotal)
                            threeDProb = p * zProb

                            row.append(sigmaTotal)
                            row.append(zProb)
                            row.append(threeDProb)

                            output_rows.append(row)
                    else:
                        continue

                writer.writerows(output_rows)

## Here we make manual cuts by 3D Probability to 20170314_GW_Resolved.csv file. Want code that does this. Want top 3000

For a good chunk of the data the probability is 0

In [2]:
from astroquery.irsa_dust import IrsaDust


In [10]:
with open("20170314_GW_Resolved_TopProb.csv","r") as csvinput:
    with open("20170314_GW_Resolved_TopProb_dust.csv","w") as csvoutput:
        
        reader = csv.reader(csvinput, delimiter=",")
        writer = csv.writer(csvoutput, lineterminator="\n")
        
        output_rows = []
        header_row = next(reader)
        header_row.append("Av")
        
        output_rows.append(header_row)
        
        for row in reader:
            ra = row[0] # RA (degree) col
            dec = row[1] # DEC (degree) col
            c = coord.SkyCoord(ra+dec,unit=(u.hourangle, u.deg),frame="fk5")
            
            dust_table = IrsaDust.get_extinction_table(c)
            av = dust_table["A_SandF"][np.where(dust_table["Filter_name"] == "CTIO V")][0]
            row.append(av)
            output_rows.append(row)

        writer.writerows(output_rows)

## Take top targets that have dust less than 0.5 from file 20170314_GW_Resolved_TopProb_dust.csv.... again manually


In [4]:
from matplotlib.pyplot import cm 
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
from cycler import cycler
import datetime
import time

try:
    from itertools import map
except ImportError:
    # Python 3...
    imap=map
from operator import itemgetter

In [12]:
with open('20170314_GW_Resolved_TopProb_TopDust.csv', 'r') as input_file:
     with open('.csv', 'w') as output_file:
        reader = csv.reader(input_file, delimiter=",")
        writer = csv.writer(output_file, lineterminator="\n")
            
        writer.writerows(imap(itemgetter(3, 0, 1, 26), reader))

In [ ]:
with open('20170314_GW_Resolved_TopProb_TopDust.csv', 'r') as csvinput:
    with open('20170314_GW_Resolved_TopProb_TopDust_Mag.csv', 'w') as csvoutput:
            
        reader = csv.reader(csvinput, delimiter=",")   
        writer = csv.writer(csvoutput, lineterminator="\n")
        
        output_rows = []
        header_row = next(reader)                    #This is adding the apparent Mag and Date columns
        header_row.append("App_Mag")
        header_row.append("Disc_Date")
        output_rows.append(header_row)
        
        for row in reader:
            row.append(18.)              #Mag 18
            row.append(20170314)         #Date
            output_rows.append(row)

        writer.writerows(output_rows)


make code that adds the type, exposure time, and absolute magnitude columns.
Manually calculate priority which is the inverse of the probability.